# Проект реккомендации фильмов

### Описание проекта:
Проект представляет собой интеллектуальную рекомендательную систему, которая помогает пользователям открывать для себя новые фильмы на основе их предпочтений и популярности кинокартин. Система предлагает рекомендации, учитывая такие факторы, как жанры, рейтинги, популярность, описание фильмов и ключевые слова.

### Описание данных:

* #### id <br>
Уникальный идентификатор фильма. (тип: int)
* #### title <br>
Название фильма. (тип: str)
* #### vote_average <br>
Средняя оценка фильма зрителями. (тип: float)
* #### vote_count <br>
Общее количество оценок фильма. (тип: int)
* #### status <br>
Статус фильма (например, "Выпущен", "В разработке", "Постпродакшн" и т.д.). (тип: str)
* #### release_date <br>
Дата выхода фильма. (тип: str)
* #### revenue <br>
Общий доход от фильма. (тип: int)
* #### runtime <br>
Продолжительность фильма в минутах. (тип: int)
* #### adult <br>
Указывает, предназначен ли фильм только для взрослой аудитории. (тип: bool)
* #### backdrop_path <br>
URL фонового изображения фильма. (тип: str)
* #### budget <br>
Бюджет фильма. (тип: int)
* #### homepage <br>
URL официального сайта фильма. (тип: str)
* #### imdb_id <br>
IMDB ID фильма. (тип: str)
* #### original_language <br>
Язык оригинала, на котором снят фильм. (тип: str)
* #### original_title <br>
Оригинальное название фильма. (тип: str)
* #### overview <br>
Краткое описание сюжета фильма. (тип: str)
* #### popularity <br>
Популярность фильма (числовой показатель). (тип: float)
* #### poster_path <br>
URL постера фильма. (тип: str)
* #### tagline <br>
Слоган фильма. (тип: str)
* #### genres <br>
Список жанров фильма (в виде строки). (тип: str)
* #### production_companies <br>
Список кинокомпаний, участвовавших в производстве (в виде строки). (тип: str)
* #### production_countries <br>
Список стран-производителей (в виде строки). (тип: str)
* #### spoken_languages <br>
Список языков, на которых говорят в фильме (в виде строки). (тип: str)
* #### keywords <br>
Ключевые слова, связанные с фильмом (используйте .split(", ") для преобразования в список). (тип: str)

### В чем состоит задача?

В этом проекте я попытаюсь реализовать несколько алгоритмов рекомендаций. Также улучшу один из алгоритмов, чтобы пользователь мог получить рекомендацию не по одному фильму, который он введет, а до трех вводов.

### Загрузка библиотек и данных

In [8]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

import os
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
RND_ST = 12345

Расширение рабочего пространства

In [10]:
from IPython.core.display import display, HTML
display(HTML("""
<style>
.container { 
   width:90% !important; 
   position: relative; 
   right: 25px; 
}
</style>
"""))

C:\Users\temoc\AppData\Local\Temp\ipykernel_14544\2057739371.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Загружаем датасеты, проверяя что путь верный

In [12]:
def pth_exist(pth):
    if os.path.exists(pth):
        df = pd.read_csv(pth)
        return df
    else:
        print('Путь не таков')
        return None

In [13]:
df = pth_exist(r'C:\Users\temoc\OneDrive\Рабочий стол\kaggle\reccom\data\TMDB_movie_dataset_v11.csv')

Делаем функцию для выдачи общей информации по датасету

In [15]:
def info_func(df):
    print('Общая информация')
    print('')
    df.info()
    print('')
    print(df.describe())
    print('')
    print('Кол-во пропущенных значений')
    print('')
    print(df.isna().sum().sort_values(ascending=False))
    print('')
    print('Процент пропущенных значений')
    print('')
    print((df.isna().sum()/len(df)*100).sort_values(ascending=False))
    print('')
    print('Как выглядит датасет')
    print('')
    print(df.head(10))

Выводим общую информацию

In [17]:
info_func(df)

Общая информация

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151022 entries, 0 to 1151021
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1151022 non-null  int64  
 1   title                 1151009 non-null  object 
 2   vote_average          1151022 non-null  float64
 3   vote_count            1151022 non-null  int64  
 4   status                1151022 non-null  object 
 5   release_date          957813 non-null   object 
 6   revenue               1151022 non-null  int64  
 7   runtime               1151022 non-null  int64  
 8   adult                 1151022 non-null  bool   
 9   backdrop_path         304206 non-null   object 
 10  budget                1151022 non-null  int64  
 11  homepage              121619 non-null   object 
 12  imdb_id               606248 non-null   object 
 13  original_language     1151022 non-null  object 
 14  original_title  

Сильно ограничиваем наш изначальный датасет, убираем все "взрослые" фильмы, берем все строки, где нет пропусков для ключевых столбцов, а также берем только 2/100 от общего объема датасета (из-за вычислительных мощностей) получим df_first для реализации Content Based Recommender. Датасет df_second, который имеет 50% от объема изначального датасета используется для Simple Recommender с меньшими вычислительными затратами.

In [19]:
df_first = df[
    (df['vote_average'] > 0) & 
    (~df['adult']) & 
    (df['keywords'].notna()) & 
    (df['tagline'].notna()) & 
    (df['genres'].notna()) &
    (df['vote_average'] > 7.0)
].sample(frac=0.02, random_state=RND_ST)

In [20]:
df_second = df[
    (df['vote_average'] > 0) & 
    (~df['adult']) & 
    (df['keywords'].notna()) & 
    (df['tagline'].notna()) & 
    (df['genres'].notna()) &
    (df['vote_average'] > 7.0)
].sample(frac=0.5, random_state=RND_ST)

Посмотрим как выглядит датасет

In [22]:
df_first.head(10)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
678,782,Gattaca,7.548,5786,Released,1997-09-07,12532777,106,False,/hPsCR1ny6GnctJkWqeJwihTDD7T.jpg,...,Gattaca,In a future society in the era of indefinite e...,25.598,/eSKr5Fl1MEC7zpAXaLWBWSBjgJq.jpg,There is no gene for the human spirit.,"Thriller, Science Fiction, Mystery, Romance","Jersey Films, Columbia Pictures",United States of America,"English, Esperanto","genetics, cheating, paraplegic, suicide attemp..."
80618,609598,The Day I Died: Unclosed Case,7.111,9,Released,2020-11-12,0,116,False,/AvicZEs8xf4UrrpXHH2YURqm4tO.jpg,...,내가 죽던 날,The detective Hyun-soo tries to track down a g...,2.996,/s1yoGrSWRQDi1At16tnkNTBVl89.jpg,My life has begun again.,"Drama, Crime, Mystery","Warner Bros. Korea, Oscar 10 Studio, 스토리퐁",South Korea,NaN,detective
197956,380671,Sunset Rock,7.500,2,Released,2016-02-01,0,95,False,NaN,...,Sunset Rock,"A coming-of-age film about celebrity worship, ...",0.692,/86h0BceXMrVeM6XxQ9xIVB29K6h.jpg,Everyone is a fan. Everyone is a creator. Ever...,"Comedy, Drama, Romance",Suburban Skies Pictures,NaN,English,"road trip, coming of age, internet, hollywood,..."
9576,8429,Paisan,7.596,272,Released,1946-12-10,0,125,False,/zNjp6eolIcpGx11tDbE5eekeHEG.jpg,...,Paisà,Six vignettes follow the Allied invasion from ...,13.242,/r5IGAFASCXmp8m51vUbER3WwVcB.jpg,There are always opportunities for redemption.,"Drama, War","Organizzazione Film Internazionali (OFI), Fore...",Italy,"German, English, Italian","rome, italy, sicily, italy, naples, italy, flo..."
291490,654429,Gloria,10.000,1,Released,2019-11-26,0,19,False,NaN,...,Glória,Glória is taken by a man to a phobia treatment...,0.925,/jcxZ60j1XUVUQhIDjHRTKIuDTBF.jpg,Short film inspired by Yorgos Lanthimos' work.,"Comedy, Drama",NaN,Brazil,Portuguese,"medo, yorgos lanthimos"
185472,492442,Now Playing,7.500,2,Released,2015-08-20,0,91,False,/dJ8k8f37YwNfiMNyfzFgwAG0Fvr.jpg,...,오늘영화,You and my romance 'Now Playing' screening beg...,0.964,/i4lYP0gQTlPxbgEHHb4s3s9g6i9.jpg,Do we want to see a movie today?,"Drama, Romance",Indieplug,South Korea,Korean,episodes
12960,40149,Kuroneko,7.398,171,Released,1968-02-24,0,100,False,/8cgoNoDsuOJOz9teAoBw6vO6Hrt.jpg,...,藪の中の黒猫,"In the Sengoku period, a woman and her daughte...",7.292,/43LZYSBRMZoi9QdYntkGXhLSMeI.jpg,Beware the haunted women who lurk in the bambo...,"Horror, Fantasy",TOHO,Japan,Japanese,"samurai, revenge, murder, black cat, ghost, ra..."
1225,6145,Fracture,7.277,3599,Released,2007-04-19,91354215,113,False,/mujUrk2diGe5vRCb3kdpHZeobRs.jpg,...,Fracture,A husband is on trial for the attempted murder...,23.784,/qNen8x5gaikjIg9CFihgxYcJwQe.jpg,I shot my wife... prove it.,Thriller,"M7 Filmproduktion, New Line Cinema, Castle Roc...","Germany, United States of America",English,"perfect crime, prosecution, legal thriller"
1909,713704,Evil Dead Rise,7.006,2292,Released,2023-04-12,146733054,96,False,/7bWxAsNPv9CXHOhZbJVlj2KxgfP.jpg,...,Evil Dead Rise,A reunion between two estranged sisters gets c...,188.126,/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg,Mommy loves you to death.,"Thriller, Horror","New Line Cinema, Ghost House Pictures, Renaiss...","Ireland, New Zealand, United States of America",English,"sibling relationship, pregnancy, earthquake, g..."
120365,323888,Storm in the Andes,7.200,5,Released,2015-02-06,0,101,False,NaN,...,Storm över Anderna,Josephine has all her life been told that her ...,0.804,/uYZShi2HqPtWjbAFZqx0wbDcqyU.jpg,They meet over an abyss,Documentary,Månharen Film & TV AB,Sweden,Spanish,"peru, sweden, militant, poor"


### Content Based Recommender

Чтобы сделать рекомендации более персонализированными, я собираюсь создать систему, которая вычисляет схожесть между фильмами на основе определённых метрик и предлагает фильмы, наиболее похожие на те, которые понравились пользователю. Поскольку мы будем использовать метаданные фильмов (или их содержание) для построения этой системы, то такой подход известен как контентная фильтрация (Content Based Filtering).

Я создам две рекомендательные системы на основе контента, используя:

1. Описания фильмов и слоганы.

2. Ключевые слова и жанры.

Кроме того, как упоминалось во введении, я буду использовать подмножество всех доступных фильмов из-за ограниченных вычислительных ресурсов.

Сначала попробуем построить рекомендательную систему, используя описания фильмов и их слоганы.

У нас нет количественного показателя для оценки производительности нашей системы, поэтому это придётся делать качественно (на основе субъективной оценки)

In [27]:
df_first['tagline'] = df_first['tagline'].fillna('')
df_first['description'] = df_first['overview'] + " " + df_first['tagline']
df_first['description'] = df_first['description'].fillna('')

Переведем наше description в матрицу признаков TF-IDF

In [29]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.01, stop_words='english')
tf_matrix = tf.fit_transform(df_first['description'])

In [30]:
tf_matrix.shape

(237, 526)

Я буду использовать косинусное сходство для вычисления числовой величины, которая обозначает степень схожести между двумя фильмами.
 
Поскольку мы использовали TF-IDF векторизацию, вычисление скалярного произведения напрямую даст нам значение косинусного сходства. Поэтому я буду использовать функцию linear_kernel из библиотеки sklearn, а не cosine_similarities, так как она работает значительно быстрее.

In [32]:
cos_sim = linear_kernel(tf_matrix, tf_matrix)

In [33]:
cos_sim[0]

array([1.        , 0.05681751, 0.07999919, 0.        , 0.        ,
       0.        , 0.03070074, 0.        , 0.        , 0.02952589,
       0.        , 0.        , 0.        , 0.02461002, 0.        ,
       0.        , 0.03538289, 0.54843047, 0.        , 0.06583657,
       0.        , 0.        , 0.08403636, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.07027311, 0.        , 0.        , 0.        , 0.01966587,
       0.        , 0.        , 0.        , 0.        , 0.04982009,
       0.03237078, 0.        , 0.        , 0.10538212, 0.16685369,
       0.        , 0.        , 0.        , 0.        , 0.02766584,
       0.        , 0.        , 0.07696477, 0.        , 0.02690594,
       0.        , 0.        , 0.02542337, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.06396189,
       0.04358912, 0.08816894, 0.        , 0.        , 0.        ,
       0.        , 0.05270306, 0.05420276, 0.        , 0.     

Теперь у нас есть попарная матрица косинусного сходства для всех фильмов в нашем наборе данных. Следующий шаг — написать функцию, которая возвращает 30 наиболее похожих фильмов на основе оценки косинусного сходства.

In [35]:
df_first = df_first.reset_index()
titles = df_first['title']
indices = pd.Series(df_first.index, index=df_first['title'])

In [36]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Испытаем нашу функцию (фильм для проверки есть в  наборе)

In [38]:
get_recommendations('Mission: Impossible - Dead Reckoning Part One').head(10)

27     Kamen Rider Amazons The Movie: The Final Judgment
130                                              Silmido
80                                   Saving Private Ryan
131                                            Deathgrip
96                                         film d'auteur
127                                  Lucky Number Slevin
56                                       The Fisher King
233                                  I Don't Fire Myself
73                                                Liyana
45                               Giantess Battle Attack!
Name: title, dtype: object

Из-за того, что у нас достаточно ограниченный датасет - то в датасет могли не попасть популярные фильмы

Для создания рекомендательной системы, добавим keywords и credits. Подготовим данные.

Прежде чем использовать ключевые слова, мы проведём их небольшую предварительную обработку. Первым шагом мы вычислим частоту встречаемости каждого ключевого слова в наборе данных.

In [42]:
keywords = df_first['keywords'].value_counts()
keywords[:10]

keywords
wrestling                                                                                                                                                                                                                     2
woman director                                                                                                                                                                                                                2
genetics, cheating, paraplegic, suicide attempt, hostility, dystopia, dna, new identity, investigation, heart disease, spaceman, fake identity, blood sample, biotechnology, space mission, exercise, eugenics                1
london, england, sibling relationship, based on novel or book, parent child relationship, magic, famous score, nanny, musical, family, kite flying, live action and animation, 1910s, chimney sweep, suffragettes, adoring    1
cheating, darkness, betrayal, black and white, crime syndicate, rock paper scissors, short film

Приведем все слова к начальной форме

In [44]:
stemmer = SnowballStemmer('english')

In [45]:
df_first['keywords'] = df_first['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [46]:
df_first['keywords'] = df_first['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

Объединим ключевые слова и жанры

In [48]:
df_first['all'] = df_first.apply(lambda row: ' '.join(row['keywords']) + ' '.join(row['genres']), axis=1)
df_first['all'] = df_first['all'].apply(lambda x: ' '.join(x))

In [49]:
print(df_first['all'].head(10))
print(df_first['all'].str.len().describe())

0    g   e   n   e   t   i   c   s   ,     c   h   ...
1    d   e   t   e   c   t   i   v   e D   r   a   ...
2    r   o   a   d     t   r   i   p   ,     c   o ...
3    r   o   m   e   ,     i   t   a   l   y   ,   ...
4    m   e   d   o   ,     y   o   r   g   o   s   ...
5    e   p   i   s   o   d   e   s D   r   a   m   ...
6    s   a   m   u   r   a   i   ,     r   e   v   ...
7    p   e   r   f   e   c   t     c   r   i   m   ...
8    s   i   b   l   i   n   g     r   e   l   a   ...
9    p   e   r   u   ,     s   w   e   d   e   n   ...
Name: all, dtype: object
count     237.000000
mean      354.907173
std       282.483133
min        39.000000
25%       147.000000
50%       253.000000
75%       475.000000
max      1399.000000
Name: all, dtype: float64


In [50]:
df_first['all'] = df_first['all'].str.replace(r'\s+', '', regex=True)

print(df_first['all'].head(10))

0    genetics,cheating,paraplegic,suicideattempt,ho...
1                         detectiveDrama,Crime,Mystery
2    roadtrip,comingofage,internet,hollywood,summer...
3    rome,italy,sicily,italy,naples,italy,florence,...
4                     medo,yorgoslanthimosComedy,Drama
5                                episodesDrama,Romance
6    samurai,revenge,murder,blackcat,ghost,rapeandr...
7       perfectcrime,prosecution,legalthrillerThriller
8    siblingrelationship,pregnancy,earthquake,gore,...
9                 peru,sweden,militant,poorDocumentary
Name: all, dtype: object


Мы создали "дамп метаданных" для каждого фильма, который включает жанр, описание и ключевые слова. Используем Count Vectorizer для создания матрицы частот, и вычисляем косинусное сходство и возвращаем наиболее похожие фильмы.

In [52]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.01, stop_words=None)
count_matrix = count.fit_transform(df_first['all'])

In [53]:
print(count.vocabulary_)

{'cheating': 16, 'dystopia': 32, 'sciencefiction': 72, 'mystery': 59, 'romance': 69, 'crime': 22, 'roadtrip': 68, 'comingofage': 19, 'drama': 28, 'drama romance': 30, 'italy': 48, 'war': 85, 'revenge': 66, 'murder': 55, 'fantasy': 35, 'revenge murder': 67, 'siblingrelationship': 75, 'sequel': 73, 'losangeles': 51, 'california': 15, 'horror': 45, 'losangeles california': 52, 'dream': 31, 'nightmare': 60, 'transformation': 83, 'food': 36, 'thriller': 80, 'animation': 5, 'comedy': 18, 'worldwarii': 87, 'history': 43, 'wrestling': 88, 'action': 1, 'gangster': 38, 'musical': 58, 'basedonnovelorbook': 9, 'blackandwhite': 13, 'superhero': 77, 'adventure': 2, 'concert': 20, 'relationship': 65, 'basedontruestory': 11, 'airplane': 3, 'courtroom': 21, 'parentchildrelationship': 61, 'health': 40, 'prison': 64, 'crime drama': 23, 'music': 56, 'tvmovie': 84, 'daughter': 24, 'music romance': 57, '1980s': 0, 'texas': 79, 'alcoholic': 4, 'lossoflovedone': 53, 'family': 34, 'husbandwiferelationship': 46

In [54]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [55]:
df_first= df_first.reset_index()
titles = df_first['title']
indices = pd.Series(df_first.index, index=df_first['title'])

In [56]:
get_recommendations('Mission: Impossible - Dead Reckoning Part One').head(10)

27     Kamen Rider Amazons The Movie: The Final Judgment
130                                              Silmido
80                                   Saving Private Ryan
131                                            Deathgrip
96                                         film d'auteur
127                                  Lucky Number Slevin
56                                       The Fisher King
233                                  I Don't Fire Myself
73                                                Liyana
45                               Giantess Battle Attack!
Name: title, dtype: object

Из-за того, что у нас достаточно маленькая выборка, а также нету имен режиссера и актеров - довольно трудно сказать работает ли наша система в целом. Большую часть фильмов я здесь не узнаю - это одна из проблем этого датасета, что здесь БУКВАЛЬНО все фильмы до 2024 года включительно. И выбрать какие фильмы войдут в такую (2/100 от реального размера) выборку - трудно предсказать. Тем не менее Счастливое число Слевина - интересный выбор. Имеются экшн сцены, и thrill (жанр триллер) от просмотра. Примерно то же самое и с Спасти рядового Райна, но я бы не стал советовать его тому кто выбрал Миссию Невыполнима - фильмы довольно разные по содержанию.

Одна из особенностей нашей рекомендательной системы заключается в том, что она рекомендует фильмы независимо от их рейтингов и популярности. Поэтому мы добавим механизм, который исключит плохие фильмы и будет рекомендовать только те, которые популярны и получили хорошие отзывы критиков.

Я использую рейтинги TMDB для создания нашего чарта лучших фильмов. Для построения чарта я буду использовать формулу взвешенного рейтинга IMDB. Где:

v — количество голосов за фильм,

m — минимальное количество голосов, необходимое для попадания в чарт,

R — средний рейтинг фильма,

C — средний рейтинг всех фильмов в отчёте.

In [60]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

Следующий шаг — определить подходящее значение для m.
Мы будем использовать 95-й процентиль в качестве порога. Другими словами, чтобы попасть в чарт, фильм должен иметь больше голосов, чем как минимум 95% фильмов в списке.

In [62]:
m = df_first['vote_count'].quantile(0.95) 

In [63]:
C = df_first['vote_average'].mean() 

In [64]:
df_first['year'] = pd.to_datetime(df_first['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [65]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = df_first.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('float')
    
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    
    qualified = movies[
        (movies['vote_count'] >= m) &
        (movies['vote_count'].notnull()) &
        (movies['vote_average'].notnull())
    ].copy()
    
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('float')
    
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [66]:
improved_recommendations('Mission: Impossible - Dead Reckoning Part One')

,title,vote_count,vote_average,year,wr
26,Woman in the Dunes,358,8.285,1964,8.175790
76,Call Me by Your Name,11367,8.167,2017,8.166662
74,Porco Rosso,2922,7.787,1992,8.002568
211,Amores Perros,2335,7.626,2000,7.962323
63,In the Mouth of Madness,1465,7.216,1995,7.904495
65,Ong-Bak,1548,7.089,2003,7.857593
127,Lucky Number Slevin,3740,7.479,2006,7.827843
8,Evil Dead Rise,2292,7.006,2023,7.733760
79,Taken,10432,7.387,2008,7.597383
181,The Maze Runner,15901,7.173,2014,7.366983


Наши рекомендации явно изменились, я узнаю гораздо больше фильмов и они действительно могли бы подойти в качестве рекомендации. Счастливое число Слевина осталось, а вот Рядовой Райан спасен не был, оно и к лучшему. Правда почему то нам теперь советуют Назови меня своим именем - который совершенно не подходит нам по жанрам, но подозреваю, что это по тем же причинам, что я назвал ранее.

### Content Based Recommender, но для трех фильмов

Создадим функцию, на основе предыдущей, в которую можно ввести до трех фильмов

In [69]:
def improved_recommendations_multiple(titles):
    # Проверка, что введён хотя бы один фильм
    if len(titles) == 0 or len(titles) > 3:
        raise ValueError("Введите от одного до трёх названий фильмов.")
    
    # Получение индексов для всех введённых фильмов
    indices_list = [indices[title] for title in titles if title in indices]
    
    if not indices_list:
        raise ValueError("Ни один из введённых фильмов не найден в базе данных.")
    
    # Вычисление суммарного сходства для всех введённых фильмов
    sim_scores = sum([list(enumerate(cosine_sim[idx])) for idx in indices_list], [])
    
    # Группировка сходства по индексу и суммирование их значений
    from collections import defaultdict
    combined_scores = defaultdict(float)
    for idx, score in sim_scores:
        combined_scores[idx] += score
    
    # Сортировка фильмов по комбинированным сходствам
    sorted_scores = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    sorted_scores = sorted_scores[len(titles):26]  # Исключение исходных фильмов и ограничение на топ-25
    
    movie_indices = [i[0] for i in sorted_scores]
    
    # Извлечение данных о фильмах
    movies = df_first.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    
    # Расчёт среднего рейтинга и порога голосов
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('float')
    
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    
    # Отбор фильмов с достаточным количеством голосов и хорошими оценками
    qualified = movies[
        (movies['vote_count'] >= m) &
        (movies['vote_count'].notnull()) &
        (movies['vote_average'].notnull())
    ].copy()
    
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('float')
    
    # Расчёт взвешенного рейтинга
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    
    # Сортировка по рейтингу и возврат топ-10 фильмов
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [70]:
improved_recommendations_multiple(['Mission: Impossible - Dead Reckoning Part One', 'Saving Private Ryan', 'Fracture' ])

,title,vote_count,vote_average,year,wr
26,Woman in the Dunes,358,8.285,1964,8.175790
93,Masquerade,135,7.700,2012,8.149939
3,Paisan,272,7.596,1946,8.128185
35,Airlift,176,7.116,2016,8.119916
211,Amores Perros,2335,7.626,2000,7.962323
197,The Zookeeper's Wife,1414,7.363,2017,7.950547
63,In the Mouth of Madness,1465,7.216,1995,7.904495
65,Ong-Bak,1548,7.089,2003,7.857593
79,Taken,10432,7.387,2008,7.597383


Как и ранее, трудно сказать работает ли наша система в целом, но я посмотрел описание фильма Airlift - и фильм действительно мог бы претендовать на то, чтобы 'подходить' под каждый из трех фильмов. Далее я хочу создать Простую рекомендательную систему - для того, чтобы пользователь мог посмотреть одни из самых популярных фильмов, если он не хочет чего-то определенного.

# Реализация Simple Recommender

Простая рекомендательная система предлагает общие рекомендации для всех пользователей на основе популярности фильмов и (иногда) жанра. Основная идея этой системы заключается в том, что фильмы, которые более популярны и получили большее признание критиков, с большей вероятностью понравятся среднестатистическому зрителю. Эта модель не предоставляет персонализированных рекомендаций, основанных на предпочтениях конкретного пользователя.

Реализация этой модели крайне проста. Всё, что нужно сделать, — это отсортировать фильмы по рейтингу и популярности, а затем показать топ фильмов из списка. В качестве дополнительного шага можно передать аргумент жанра, чтобы получить лучшие фильмы определённого жанра.

Для начала попробуем создать просто топ фильмов независимо от жанров. Как и ранее мы будем использовать 95-й процентиль в качестве порога.

In [75]:
df_second['year'] = pd.to_datetime(df_second['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [76]:
qualified = df_second[(df_second['vote_count'] >= m) & (df_second['vote_count'].notnull()) & (df_second['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(300, 6)

In [77]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [78]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [79]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
1091,Once Upon a Time in the West,1968,3923,8,48.342,"Drama, Western",8.082184
1081,Cinema Paradiso,1988,3954,8,30.498,"Drama, Romance",8.081858
864,Once Upon a Time in America,1984,4863,8,42.059,"Drama, Crime",8.073330
787,Casino,1995,5240,8,29.212,"Crime, Drama",8.070293
614,Lion,2016,6199,8,20.266,Drama,8.063593
575,Puss in Boots: The Last Wish,2022,6394,8,221.490,"Animation, Family, Fantasy, Adventure, Comedy,...",8.062383
504,Ford v Ferrari,2019,6972,8,78.353,"Drama, Action, History",8.059055
461,Pride & Prejudice,2005,7394,8,49.413,"Drama, Romance",8.056841
452,Apocalypse Now,1979,7485,8,37.715,"Drama, War",8.056385
444,Top Gun: Maverick,2022,7546,8,126.291,"Action, Drama",8.056084


Довольно известные фильмы, я вполне доволен результатом.

Давайте теперь создадим функцию, которая будет строить чарты для определённых жанров. Для этого мы ослабим наши стандартные условия и будем использовать 85-й процентиль вместо 95-го. Так как у нас будет более конкретная выборка по жанрам.

In [82]:
s = df_second.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_df_second = df_second.drop('genres', axis=1).join(s)

In [83]:
def build_chart(genre, percentile=0.85):
    df = gen_df_second[gen_df_second['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [84]:
build_chart('Horror').head(15)

,title,year,vote_count,vote_average,popularity,wr
141339,TASTY!,2023,4,9,0.600,9.000000
137025,The Exorcist's Vengeful Curse,2001,4,8,0.774,8.500000
95408,The Woods,2017,7,7,1.551,7.727273
54661,Toby Dammit,1969,17,7,1.545,7.380952
43703,Jikirag,2022,24,7,10.073,7.285714
8561,Don't Torture a Duckling,1972,323,7,11.294,7.024465
8005,Black Sabbath,1963,358,7,10.778,7.022099
6903,Vampyr,1932,451,7,12.555,7.017582
6846,Black Sunday,1960,457,7,17.671,7.017354
6151,The Changeling,1980,531,7,20.266,7.014953


Вероятно из-за того, что у нас 50% от реального размера датасета - сюда не вошли джругие популярные хорроры, но даже в этом списке есть действительно известные фильмы, напрмиер Техасская Резня Бензопилой.

Следующим моим шагом будет реализация этих моделей в реальных условиях с помощью fastapi.